In [16]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

In [17]:
# read in csv file
all_cases = gpd.read_file(r'C:/Users/olubl/OneDrive/College/Data+/Durham Development Data.zip')
all_cases.head()

,A_NUMBER,A_TYPE,A_DATE,A_STATUS,A_STATUS_D,A_PROJECT_,A_DESCRIPT,A_USER_ID,A_CASE_PLA,StatCode,...,AppCode,AppType,CasePlanne,EMAIL,ORIG_FID,CreationDa,Creator,EditDate,Editor,geometry
0,S1900405,FIN_EXE,2019-11-26,APP,2019-12-13,Longebach Recombination Survey,Recombine two lots into 1,BRYANTGR,MICHAELKL,APP,...,FIN_EXE,Exempt Plat,Michael Klein,Michael.Klein@durhamnc.gov,1,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2.02e+06 8.58e+05)
1,HO1400256,PL_HOP,2014-05-06,APP,2014-05-06,Proserv Cleaner,12 Trescott Drive is the home office for the c...,MARIAHER,HSMITH,APP,...,PL_HOP,Home Occupation Permit,None,None,2,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2.06e+06 8.08e+05)
2,S1700288,FIN_EXE,2017-12-11,APP,2018-01-16,Brightleaf at the Park Tract 7 Easement,None,JESSICADO,JESSICADO,APP,...,FIN_EXE,Exempt Plat,Jessica Dockery,Jessica.Dockery@durhamnc.gov,3,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2.05e+06 8.01e+05)
3,HO1100444,PL_HOP,2011-07-29,APP,2011-07-29,Fam or Foe Ent,Home office for musician - no onsite shows or ...,DDOTY,DDOTY,APP,...,PL_HOP,Home Occupation Permit,Dennis Doty,Dennis.Doty@durhamnc.gov,4,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2.04e+06 8.13e+05)
4,S1400114,FIN_MAJ,2014-06-24,APP,2015-07-29,Phillip and Gina Ellis Plat,Minor Subdivision,AMANDAHO,JENNIFERL,APP,...,FIN_MAJ,Final Plat,Jennifer Leaf,Jennifer.Leaf@durhamnc.gov,5,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2.05e+06 9.02e+05)


In [18]:
# types = A_TYPE codes we want to keep
types = ['PL_MINSP', 'PL_SSP_SM', 'PL_SSM_SM2', 'PL_MJ_COA', 'D_REZ', 'PL_CPAA', 'PL_MINPP', 'PL_MAJSP', 'PL_MAJSUP', 'PL_PPA', 'PL_MAJPP'] 
# filter that only keeps cases of specified types
filter_cases_type = all_cases[all_cases['A_TYPE'].isin(types)]

In [53]:
# status = A_STATUS codes we want to keep
status = all_cases['A_STATUS'].unique()
status = status[~np.isin(status, ['WITH', 'VOID','DEN','DISAP','EXP'])]
# filter that only keeps cases of specified status
filter_cases_status = filter_cases_type[filter_cases_type['A_STATUS'].isin(status)]

In [20]:
# filter_cases_status

In [54]:
keywords = ['home', 'family', 'residen', 'mixed', 'mized', 'duplex', 'apartment', ' housing', 'condo', 'dwelling', 'tenant', 'affordable', 'units', 'townhouse']
pattern = '|'.join(keywords)
filtered1 = filter_cases_status[filter_cases_status['A_DESCRIPT'].str.contains(pattern, case=False, na=False)]
#use not pattern to avoid these words
keywords_avoid = ['addition','expand','storage']
pattern_avoid = '|'.join(keywords_avoid)
filtered2 = filtered1[~filtered1['A_DESCRIPT'].str.contains(pattern_avoid, case=False, na=False)]

In [55]:
#filter for all status dates after 2020
filtered2['A_STATUS_D'] = pd.to_datetime(filtered2['A_STATUS_D'])
filtered3 = filtered2[filtered2['A_STATUS_D'].dt.year>=2020]
filtered3

C:\Users\olubl\anaconda3\envs\spatialdata\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,A_NUMBER,A_TYPE,A_DATE,A_STATUS,A_STATUS_D,A_PROJECT_,A_DESCRIPT,A_USER_ID,A_CASE_PLA,StatCode,...,AppCode,AppType,CasePlanne,EMAIL,ORIG_FID,CreationDa,Creator,EditDate,Editor,geometry
90,D2000291,PL_MINSP,2020-12-02,APP,2021-09-03,Umstead Grove Conservation Subdivision,"50 Single - family lots, 1 stormwater pond, ad...",JESSICADO,COURTNEYMC,APP,...,PL_MINSP,Level 4 - Administrative Site Plan,Courtney McQueen,Courtney.McQueen@durhamnc.gov,91,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2.02e+06 8.46e+05)
139,D2200173,PL_MINSP,2022-05-25,APP,2023-02-01,Pineview Glen Town homes - Mass Grading,Mass grading only site plan for residential to...,JUSTINH,KEAGANSA,APP,...,PL_MINSP,Level 4 - Administrative Site Plan,Keagan Sacripanti,Keagan.Sacripanti@durhamnc.gov,140,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2.06e+06 7.93e+05)
224,D1800378,PL_MINSP,2018-11-28,APP,2020-04-14,Elan Innovation District,Mixe of uses including retail and residential....,JOHNRA,TREYFI,APP,...,PL_MINSP,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,225,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2.03e+06 8.19e+05)
242,D2100156,PL_MINSP,2021-06-04,APP,2022-02-02,ALTA Rutherford,Multifamily Apartments,KIMRO,TREYFI,APP,...,PL_MINSP,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,243,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2.02e+06 8.23e+05)
265,D1900171,PL_MINSP,2019-05-30,APP,2020-04-03,Ellis Road Phase 3,"37 Townhome units, 102 Attached S-F units (cal...",ROBINSH,COLERE,APP,...,PL_MINSP,Level 4 - Administrative Site Plan,Cole Renigar,Cole.Renigar@durhamnc.gov,266,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2.04e+06 8.01e+05)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21215,Z2500014,D_REZ,2025-04-15,UN_RE,2025-04-15,Brittmore,115 maximum unit development of single family ...,CHEZNEYUP,ANDREWLE,UN_RE,...,D_REZ,Zoning Map Change,Andrew Lester,Andrew.Lester@durhamnc.gov,21216,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2e+06 7.76e+05)
21222,D2500095,PL_MINSP,2025-04-17,UN_RE,2025-04-17,4702 Farrington Rd - Grading Only,Grading only site plan for future residential ...,NKOST,PEYTONBL,UN_RE,...,PL_MINSP,Level 4 - Administrative Site Plan,PEYTON BLANCHARD,PEYTON.BLANCHARD@DURHAMNC.GOV,21223,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2e+06 7.95e+05)
21241,D2500103,PL_MINSP,2025-04-25,UN_RE,2025-04-25,Fayetteville Road MultiFamily,Phase one of overall development - single fami...,FRANCISCOM,TREYFI,UN_RE,...,PL_MINSP,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,21242,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2.01e+06 7.75e+05)
21255,COA2500038,PL_MJ_COA,2025-04-29,UN_RE,2025-04-29,1406 Fayetteville St-Add_Mod,"Signifiant work to roof, gutters, exterior of ...",FRANCISCOM,KROSENBERG,UN_RE,...,PL_MJ_COA,Major COA,Karla Rosenberg,Karla.Rosenberg@durhamnc.gov,21256,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2.03e+06 8.12e+05)


In [63]:
filtered3.iloc[455]['A_DESCRIPT']

'Signifiant work to roof, gutters, exterior of home, windows, parking lot, electrical (lights), and garage.'